# Histogram-Based Outlier Search

1. For each single feature (dimension), an univariate histogram is constructed first
    - If the feature comprises of categorical data, simple counting of the values of each category is performed and the relative frequency (height of the histogram) is computed.
    - For numerical features, two different methods can be used:
        1. _**Static bin-width histograms**_....
        the standard histogram building technique using k equal width bins over the value range. The frequency (relative amount) of samples falling into each bin is used as an estimate of the density (height of the bins)
        2. _**Dynamic bin-width histograms**_...values are sorted frst and then a fixed amount of N/k successive values are grouped into a single bin where N is the number of total instances and k the number of bins. Since the area of a bin in a histogram represents the number of observations, it is the same for all bins in our case. Because the width of the bin is defned by the first and the last value and the area is the same for all bins, the height of each individual bin can be computed. This means that bins covering a larger interval of the value range have less height and represent that way a lower density. However, there is one exception: Under certain circumstances, more than k data instances might have exactly the same value, for example if the feature is an integer and a long-tail distribution has to be estimated. In this case, our algorithm must allow to have more than N/k values in the same bin. Of course, the area of these larger bins will grow appropriately.
2. Now, for each dimension d, an individual histogram has been computed (regardless if categorical, fixed-width or dynamic-width), where the height of each single bin represents a density estimation. The histograms are then normalized such that the maximum height is 1.0. This ensures an equal weight of each feature to the outlier score.
3. height is 1.0. This ensures an equal weight of each feature to the outlier score. Finally, the HBOS of every instance p is calculated using the corresponding height of the bins where the instance is located:
    - <img
